In [3]:
import pandas as pd

dataset = pd.read_json(
    path_or_buf="/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/datasets/pure_datasets/2023-04-12_oasst_all.messages.jsonl",
    lines=True,
)
dataset[dataset["lang"] == "en"].shape

(72567, 18)

In [ ]:
dataset = pd.read_json(
    path_or_buf="/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/datasets/pure_datasets/2023-04-12_oasst_ready.trees.jsonl",
    lines=True,
)
# dataset[dataset["lang"] == "en"].shape

In [ ]:
dataset['prompt'][0]